In [ ]:
import numpy as np
import pydicom
import nibabel as nib
import os
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import label
from matplotlib.collections import LineCollection
import tensorflow as tf
from time import perf_counter
from tensorflow import keras
from sklearn.metrics import roc_curve, auc
from scipy.stats import wilcoxon

In [ ]:
def preprocess_TotalSeg(mask_path):
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    mask1=np.zeros((220,220,380))
    for i in range(380):
        mask_i=cv2.resize(mask[70:442,70:442,379-i],(220,220))
        for j in range(220):
            mask1[:,j,i]=mask_i[:,j]
    mask1=np.round(np.flip(mask1,axis=1))
    return(mask1)

def mask_into_binary(mask,index_list):
    #create another image matrix where the values within the greatest connected component of the target are 1 and the others are 0
    structure=np.ones((3,3,3),dtype=int)
    mask1=np.zeros(mask.shape)
    for index in index_list:
        mask_i=np.array(mask==index,dtype=int)
        if np.max(mask_i)==1:
            labeled, _ = label(mask_i,structure)
            array=labeled.flatten()
            array1=array[array>0]
            counts = np.bincount(array1)
            mask1+=np.array(labeled==np.argmax(counts),dtype=int)
    return(mask1)

def find_bounding_box(mask):
    k=np.max(np.max(mask,axis=2),axis=1)
    k_1s=np.array(range(len(k)))[k==1]
    k1=np.min(k_1s)
    k2=np.max(k_1s)
    k=np.max(np.max(mask,axis=2),axis=0)
    k_1s=np.array(range(len(k)))[k==1]
    k3=np.min(k_1s)
    k4=np.max(k_1s)
    k=np.max(np.max(mask,axis=1),axis=0)
    k_1s=np.array(range(len(k)))[k==1]
    k5=np.min(k_1s)
    k6=np.max(k_1s)
    return(k1,k2,k3,k4,k5,k6)

def read_pet_image(path_to_dcm_folder):
    #the input is the path to the folder containing all the DICOM slices
    #first, create a list of the names of all the files in this folders
    filenames=os.listdir(path_to_dcm_folder)
    #then create a list of filenames which are DICOM (i.e. have .dcm ending)
    dcm_filenames=[]
    for i in range(len(filenames)):
        if filenames[i][-4:]=='.dcm':
            dcm_filenames.append(filenames[i])
    #initialise lists for slices, their locations, and times
    slice_list=[]
    locations=[]
    times=[]
    #fill the lists by going through all the DICOM slices
    for i in range(len(dcm_filenames)):
        file='{}/{}'.format(path_to_dcm_folder,dcm_filenames[i])
        ds=pydicom.dcmread(file)
        locations.append(ds.ImagePositionPatient[2])
        times.append(ds.AcquisitionTime)
        if 'RescaleSlope' in ds and 'RescaleIntercept' in ds:
            slice_list.append(ds.pixel_array*ds.RescaleSlope+ ds.RescaleIntercept)
        else:
            slice_list.append(ds.pixel_array)
    #create new lists of the unique values of the acquisation times
    unique_times=np.sort(np.unique(np.array(times)))
    #initialize the image
    img=np.zeros((220,220,380))
    #find the 3D time-frames for all unique time values
    for i in range(len(unique_times)):
        locations_1=list(np.array(locations)[np.array(times)==unique_times[i]])
        slice_list_1=list(np.array(slice_list)[np.array(times)==unique_times[i]])
        merged=dict(zip(locations_1,slice_list_1))
        ordered_slice_list=[merged[loc] for loc in sorted(locations_1)]
        img_3d=np.moveaxis(np.array(ordered_slice_list),0,2)
        img_3d=np.moveaxis(img_3d,0,1)
        img_3d=np.flip(img_3d,axis=2)
        if i>0:
            k=(float(unique_times[i])-float(unique_times[i-1]))/(float(unique_times[-1])-float(unique_times[0]))
        else:
            k=(float(unique_times[i+1])-float(unique_times[i]))/(float(unique_times[-1])-float(unique_times[0]))
        img+=k*img_3d
    return(img)

def get_all_edges(bool_img):
    """
    Get a list of all edges (where the value changes from True to False) in the 2D boolean image.
    The returned array edges has he dimension (n, 2, 2).
    Edge i connects the pixels edges[i, 0, :] and edges[i, 1, :].
    Note that the indices of a pixel also denote the coordinates of its lower left corner.
    """
    edges = []
    ii, jj = np.nonzero(bool_img)
    for i, j in zip(ii, jj):
        # North
        if j == bool_img.shape[1]-1 or not bool_img[i, j+1]:
            edges.append(np.array([[i, j+1],
                                   [i+1, j+1]]))
        # East
        if i == bool_img.shape[0]-1 or not bool_img[i+1, j]:
            edges.append(np.array([[i+1, j],
                                   [i+1, j+1]]))
        # South
        if j == 0 or not bool_img[i, j-1]:
            edges.append(np.array([[i, j],
                                   [i+1, j]]))
        # West
        if i == 0 or not bool_img[i-1, j]:
            edges.append(np.array([[i, j],
                                   [i, j+1]]))

    if not edges:
        return np.zeros((0, 2, 2))
    else:
        return np.array(edges)


def close_loop_edges(edges):
    """
    Combine the edges defined by 'get_all_edges' to closed loops around objects.
    If there are multiple disconnected objects a list of closed loops is returned.
    Note that it's expected that all the edges are part of exactly one loop (but not necessarily the same one).
    """

    loop_list = []
    while edges.size != 0:

        loop = [edges[0, 0], edges[0, 1]]  # Start with first edge
        edges = np.delete(edges, 0, axis=0)

        while edges.size != 0:
            # Get next edge (=edge with common node)
            ij = np.nonzero((edges == loop[-1]).all(axis=2))
            if ij[0].size > 0:
                i = ij[0][0]
                j = ij[1][0]
            else:
                loop.append(loop[0])
                # Uncomment to to make the start of the loop invisible when plotting
                # loop.append(loop[1])
                break

            loop.append(edges[i, (j + 1) % 2, :])
            edges = np.delete(edges, i, axis=0)

        loop_list.append(np.array(loop))

    return loop_list


def plot_outlines(bool_img, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    edges = get_all_edges(bool_img=bool_img)
    edges = edges - 0.5  # convert indices to coordinates; TODO adjust according to image extent
    outlines = close_loop_edges(edges=edges)
    cl = LineCollection(outlines, **kwargs)
    ax.add_collection(cl)

def preprocess_slice(img,img_height):
    img=cv2.resize(img,(img_height,img_height))
    return img.astype('float16')

def fiveFoldCrossValidation(patientIndexes,slices,values,k):

    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    patientIndexes_test=[]
    patientIndexes_train=[]

    for i in range(len(patientIndexes)):
        if patientIndexes[i] % 5==k:
            x_test.append(slices[i])
            y_test.append(values[i])
            patientIndexes_test.append(patientIndexes[i])
        else:
            x_train.append(slices[i])
            y_train.append(values[i])
            patientIndexes_train.append(patientIndexes[i])

    x_train=np.array(x_train).astype('float16')
    y_train=np.array(y_train) 
    x_test=np.array(x_test).astype('float16')
    y_test=np.array(y_test)

    return(x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test)

#function for training the encoderCNN for classification and then predicting the test and train set
def predict(x_train,y_train,x_test,numberOfEpochs):
    
    img_height=x_train[0].shape[0]

    model = keras.Sequential([keras.layers.Conv2D(16, 3, activation='relu', input_shape=(img_height,img_height,1)),
                        keras.layers.Conv2D(16, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(32, 3, activation='relu'),
                        keras.layers.Conv2D(32, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(64, 3, activation='relu'),
                        keras.layers.Conv2D(64, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(128, 3, activation='relu'),
                        keras.layers.Conv2D(128, 3, activation='relu'),
                        keras.layers.MaxPooling2D(strides=(2, 2)),
                        keras.layers.Flatten(),
                        keras.layers.Dense(128, activation='relu'),
                        keras.layers.Dense(64, activation='relu'),
                        keras.layers.Dense(32, activation='relu'),
                        keras.layers.Dense(1, activation='sigmoid')
    ]
    )

    model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    history=model.fit(x=x_train,y=y_train,epochs=numberOfEpochs,shuffle=True)
    predictions=model.predict(x_test)[:,0]
    trainPreds=model.predict(x_train)[:,0]
    plt.plot(range(len(history.history['loss'])),history.history['loss'],color='blue')

    return([predictions,trainPreds])

#function for post-processing predictions by taking means of five consecutive predictions
def postProcess_new(predictions,patientIndexes_test):

    predictions_1=[]
    for i in range(len(predictions)):
        current=predictions[i]
        if i>0:
            if patientIndexes_test[i-1]==patientIndexes_test[i]:
                prev=predictions[i-1]
            else:
                prev=0
            if i>1:
                if patientIndexes_test[i-2]==patientIndexes_test[i]:
                    prev1=predictions[i-2]
                else:
                    prev1=0
            else:
                    prev1=0
        else:
            prev=0
            prev1=0
        if i<len(predictions)-1:
            if patientIndexes_test[i+1]==patientIndexes_test[i]:
                next=predictions[i+1]
            else:
                next=0
            if i<len(predictions)-2:
                if patientIndexes_test[i+2]==patientIndexes_test[i]:
                    next1=predictions[i+2]
                else:
                    next1=0
            else:
                next1=0
        else:
            next=0
            next1=0
        predictions_1.append(np.mean([prev1,prev,current,next,next1]))
    predictions_1=np.array(predictions_1)
    return(predictions_1)

#function for evaluating the predicted classification labels
#outputs: lists of patient-wise accuracy, sensitivity, specificity, precision, and AUC values
def evaluatePreds(predictions,y_test,patientIndexes_test,trainPreds,y_train):

    fpr, tpr, thresholds = roc_curve(y_train, trainPreds, drop_intermediate=False)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]

    acc_list=[]
    sen_list=[]
    spe_list=[]
    pre_list=[]
    auc_list=[]
    
    for i in range(len(np.unique(patientIndexes_test))):
        subPredictions=np.array(predictions[patientIndexes_test==np.unique(patientIndexes_test)[i]],dtype=float)
        subLabels=np.array(y_test[patientIndexes_test==np.unique(patientIndexes_test)[i]],dtype=int)
        negLabels=-subLabels+1
        TP = np.sum(subLabels[subPredictions>=youden])
        FN = np.sum(subLabels[subPredictions<youden])
        FP = np.sum(negLabels[subPredictions>=youden])
        TN = np.sum(negLabels[subPredictions<youden])
        if TP+TN==0:
            acc=0
        else:
           acc = (TP+TN)/(TP+TN+FN+FP) 
        if TP==0:
            sen=0
            pre=0
        else:
            sen = TP/(TP+FN)
            pre = TP/(TP+FP)
        if TN==0:
            spe=0
        else:
            spe = TN/(TN+FP)
        acc_list.append(acc)
        spe_list.append(spe)
        if TP+FN!=0:
            sen_list.append(sen)
            pre_list.append(pre)
            fpr, tpr, thresholds = roc_curve(subLabels,subPredictions,drop_intermediate=False)
            auc1=auc(fpr, tpr)
            auc_list.append(auc1)
    return(acc_list,sen_list,spe_list,pre_list,auc_list)

#function for finding the indexes of the slices predicted as positive
def findPredictedSliceIndexes(predictions,trainPreds,y_train):

    fpr, tpr, thresholds = roc_curve(y_train, trainPreds, drop_intermediate=False)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]
    sliceIndexes=np.array(predictions>youden,dtype=int)
    return(sliceIndexes)

def chooseIndices(i_1):
    i_1s=np.array(range(len(i_1)))[i_1==1]
    if i_1s[-1]-i_1s[0]<64:
        return([max(0,round((i_1s[-1]+i_1s[0])/2-32))])
    elif i_1s[-1]-i_1s[0]<128:
        return([i_1s[0],i_1s[-1]-64])
    elif i_1s[-1]-i_1s[0]<192:
        return([i_1s[0],round((i_1s[-1]+i_1s[0])/2),i_1s[-1]-64])
    else:
        if len(i_1)==220:
            return([0,55,101,156])
        if len(i_1)==380:
            return([0,63,126,189,252,315])

#the segmentation model (a standard U-Net model with some layers removed so that it suits for 64*64 data)
#input: x_train, y_train, x_test, the number of epochs, and the name
#output: predicted segmentation masks of the training set
def unet(x_train,y_train,x_test,numberOfEpochs,name):

    img_height=x_train[0].shape[0]
    if x_train[0].shape==(img_height,img_height):
        input_depth=1
    else:
        input_depth=x_train[0].shape[2]
    #U-Net model 
    inputs = tf.keras.layers.Input(shape=(img_height,img_height,input_depth))
    
    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)

    #Expansive path
    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c3)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer='adam', loss='Dice')

    history=model.fit(x=x_train,y=y_train,epochs=numberOfEpochs,shuffle=True)
    model.save('unet_{}.keras'.format(name),include_optimizer=True)
    testPredictions=model.predict(x_test)
    trainPredictions=model.predict(x_train)
    plt.plot(range(len(history.history['loss'])),history.history['loss'],color='blue')

    return testPredictions,trainPredictions

#function for computing patient-wise Dice scores
def computeDices(predictions,y_test,patientIndexes_test):

    dices3d=[]
    TP=0
    FN=0
    FP=0
    TN=0
    for i in range(len(patientIndexes_test)):
        TP+=np.sum(np.minimum(np.array(y_test[i]>=0.5,dtype=int),np.array(predictions[i][:,:,0]>=0.5,dtype=int)))
        FN+=np.sum(np.minimum(np.array(y_test[i]>=0.5,dtype=int),np.array(predictions[i][:,:,0]<0.5,dtype=int)))
        FP+=np.sum(np.minimum(np.array(y_test[i]<0.5,dtype=int),np.array(predictions[i][:,:,0]>=0.5,dtype=int)))
        TN+=np.sum(np.minimum(np.array(y_test[i]<0.5,dtype=int),np.array(predictions[i][:,:,0]<0.5,dtype=int)))
        if i<len(patientIndexes_test)-1:
            if patientIndexes_test[i]!=patientIndexes_test[i+1]:
                if 2*TP+FN+FP>0:
                    dices3d.append(2*TP/(2*TP+FN+FP))
                else:
                    dices3d.append(0)
                TP=0
                FN=0
                FP=0
                TN=0
    dices3d.append(2*TP/(2*TP+FN+FP))
    return(dices3d)

In [ ]:
indexes=[90,52,118,2,3,[10,11,12,13,14],5]
organ_names=['brain','aorta','myocardium','kidney_right','kidney_left','lungs','liver']
for i in [137,139,143,145,148,153,156,159,160,162,164,165]:
    #8,75,76,131,134
    if i<9:
        studyCode='koveri000{}'.format(i+1)
    else:
        if i<99:
            studyCode='koveri00{}'.format(i+1)
        else:
            studyCode='koveri0{}'.format(i+1)
    files=os.listdir('D:/img/koveri/Data/{}'.format(studyCode))
    mask_path='D:/img/koveri/doesNotExist'
    path_to_dcm_folder='D:/img/koveri/doesNotExist'
    for j in files:
        if 'rest_ct_segmentations.nii' in j:
            mask_path='D:/img/koveri/Data/{}/{}'.format(studyCode,j)
        if ('Rest' in j or 'rest' in j) and 'rest_ct_segmentations' not in j:
            if len(os.listdir('D:/img/koveri/Data/{}/{}'.format(studyCode,j)))==1:
                path_to_dcm_folder='D:/img/koveri/Data/{}/{}/{}'.format(studyCode,j,os.listdir('D:/img/koveri/Data/{}/{}'.format(studyCode,j))[0]) 
            else:
                path_to_dcm_folder='D:/img/koveri/Data/{}/{}'.format(studyCode,j)   
    if os.path.exists(mask_path) and os.path.exists(path_to_dcm_folder):
        mask=preprocess_TotalSeg(mask_path)
        img=read_pet_image(path_to_dcm_folder)
        for i in range(len(indexes)): 
            index=indexes[i]   
            if i!=5:
                mask1=mask_into_binary(mask,index)
            else:
                mask1=np.zeros((220,220,380))
                for j in index:
                    mask1+=mask_into_binary(mask,j)
            k1,k2,k3,k4,k5,k6=find_bounding_box(mask1)
            print(organ_names[i])
            img_sag=np.mean(img[k1:k2,max(k3-10,0):min(k4+10,219),max(k5-10,0):min(k6+10,379)],axis=0)
            img_sag=cv2.resize(img_sag,(round(2.8/1.65*img_sag.shape[1]),img_sag.shape[0]))
            mask_sag=np.array(np.max(mask1[k1:k2,max(k3-10,0):min(k4+10,219),max(k5-10,0):min(k6+10,379)],axis=0),dtype='uint8')
            mask_sag=cv2.resize(mask_sag,(round(2.8/1.65*mask_sag.shape[1]),mask_sag.shape[0]))
            plt.imshow(np.transpose(img_sag),cmap='gray')
            plot_outlines(np.transpose(mask_sag).T, lw=2, color='white')
            fig=plt.gcf()
            plt.show()
            fig.savefig('kuvat/{}_{}_sag.png'.format(organ_names[i],studyCode),bbox_inches='tight')
            img_cor=np.mean(img[max(k1-10,0):min(k2+10,219),k3:k4,max(k5-10,0):min(k6+10,379)],axis=1)
            img_cor=cv2.resize(img_cor,(round(2.8/1.65*img_cor.shape[1]),img_cor.shape[0]))
            mask_cor=np.array(np.max(mask1[max(k1-10,0):min(k2+10,219),k3:k4,max(k5-10,0):min(k6+10,379)],axis=1),dtype='uint8')
            mask_cor=cv2.resize(mask_cor,(round(2.8/1.65*mask_cor.shape[1]),mask_cor.shape[0]))
            plt.imshow(np.transpose(img_cor),cmap='gray')
            plot_outlines(np.transpose(mask_cor).T, lw=2, color='white')
            fig=plt.gcf()
            fig.savefig('kuvat/{}_{}_cor.png'.format(organ_names[i],studyCode),bbox_inches='tight')
            plt.show()
            img_trans=np.mean(img[max(k1-10,0):min(k2+10,219),max(k3-10,0):min(k4+10,219),k5:k6],axis=2)
            mask_trans=np.array(np.max(mask1[max(k1-10,0):min(k2+10,219),max(k3-10,0):min(k4+10,219),k5:k6],axis=2),dtype='uint8')
            plt.imshow(np.transpose(img_trans),cmap='gray')
            plot_outlines(np.transpose(mask_trans).T, lw=2, color='white')
            fig=plt.gcf()
            fig.savefig('kuvat/{}_{}_trans.png'.format(organ_names[i],studyCode),bbox_inches='tight')
            plt.show()

In [ ]:
empty_header=nib.Nifti1Header()
indexes=[[90],[52],[51,118,119,120,121,122],[118],[120],[2,3],[5],[10,11,12,13,14]]
i_ids=list(np.array(range(8)))+list(np.array(range(9,45)))+list(np.array(range(47,49)))+list(np.array(range(50,75)))+list(np.array(range(77,131)))+[132,133,137,139,143,145,148,153,156,159,160,162,164,165]
for i in i_ids:
    if i<9:
        studyCode='koveri000{}'.format(i+1)
    else:
        if i<99:
            studyCode='koveri00{}'.format(i+1)
        else:
            studyCode='koveri0{}'.format(i+1)
    files=os.listdir('D:/img/koveri/Data/{}'.format(studyCode))
    mask_path='D:/img/koveri/doesNotExist'
    path_to_dcm_folder='D:/img/koveri/doesNotExist'
    for j in files:
        if 'rest_ct_segmentations.nii' in j:
            mask_path='D:/img/koveri/Data/{}/{}'.format(studyCode,j)
        if ('Rest' in j or 'rest' in j) and 'rest_ct_segmentations' not in j:
            if len(os.listdir('D:/img/koveri/Data/{}/{}'.format(studyCode,j)))==1:
                path_to_dcm_folder='D:/img/koveri/Data/{}/{}/{}'.format(studyCode,j,os.listdir('D:/img/koveri/Data/{}/{}'.format(studyCode,j))[0]) 
            else:
                path_to_dcm_folder='D:/img/koveri/Data/{}/{}'.format(studyCode,j)   
    if os.path.exists(mask_path) and os.path.exists(path_to_dcm_folder):
        mask=preprocess_TotalSeg(mask_path)
        img=read_pet_image(path_to_dcm_folder)
        maski=np.zeros((220,220,380))
        for k in range(len(indexes)): 
            index_list=indexes[k]   
            mask1=mask_into_binary(mask,index_list)
            maski+=(1+k)*mask1
        new_img=nib.Nifti1Image(img,np.eye(4),empty_header)
        nib.save(new_img,'files/img{}.nii'.format(i+1))
        new_mask=nib.Nifti1Image(maski,np.eye(4),empty_header)
        nib.save(new_mask,'files/mask{}.nii'.format(i+1))
        print(i)

In [ ]:
empty_header=nib.Nifti1Header()
indexes=[[90],[52],[51,119,121,122],[118],[120],[2,3],[5],[10,11,12,13,14],[7]]
i_ids=list(np.array(range(8)))+list(np.array(range(9,44)))+list(np.array(range(47,49)))+list(np.array(range(50,75)))+list(np.array(range(77,131)))+[132,133,137,139,143,145,148,153,156,159,160,162,164,165]
for i in i_ids:
    if i<9:
        studyCode='koveri000{}'.format(i+1)
    else:
        if i<99:
            studyCode='koveri00{}'.format(i+1)
        else:
            studyCode='koveri0{}'.format(i+1)
    files=os.listdir('D:/img/koveri/Data/{}'.format(studyCode))
    mask_path='D:/img/koveri/doesNotExist'
    path_to_dcm_folder='D:/img/koveri/doesNotExist'
    for j in files:
        if 'rest_ct_segmentations.nii' in j:
            mask_path='D:/img/koveri/Data/{}/{}'.format(studyCode,j)
        if ('Rest' in j or 'rest' in j) and 'rest_ct_segmentations' not in j:
            if len(os.listdir('D:/img/koveri/Data/{}/{}'.format(studyCode,j)))==1:
                path_to_dcm_folder='D:/img/koveri/Data/{}/{}/{}'.format(studyCode,j,os.listdir('D:/img/koveri/Data/{}/{}'.format(studyCode,j))[0]) 
            else:
                path_to_dcm_folder='D:/img/koveri/Data/{}/{}'.format(studyCode,j)   
    if os.path.exists(mask_path) and os.path.exists(path_to_dcm_folder):
        mask=preprocess_TotalSeg(mask_path)
        maski=np.zeros((220,220,380))
        for k in range(len(indexes)): 
            index_list=indexes[k]   
            mask1=mask_into_binary(mask,index_list)
            maski+=(1+k)*mask1
        new_mask=nib.Nifti1Image(maski,np.eye(4),empty_header)
        nib.save(new_mask,'files/mask{}.nii'.format(i+1))
        print(i)

In [ ]:
folder_path='C:/Users/oonar/Documents/kov/files'
filenames=os.listdir(folder_path)
study_ids=[]
for i in range(len(filenames)):
    if 'img' in filenames[i]:
        number=int(filenames[i][3:-4])
        study_ids.append(number)
j=0
organ_names=['new']
for i in range(len(study_ids)):
    img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
    mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
    img=nib.load(img_path)
    img=img.get_fdata()
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    mask1=np.array(mask==9,dtype=int)
    k1,k2,k3,k4,k5,k6=find_bounding_box(mask1)
    img_sag=np.mean(img[k1:k2,max(k3-10,0):min(k4+10,219),max(k5-10,0):min(k6+10,379)],axis=0)
    img_sag=cv2.resize(img_sag,(round(2.8/1.65*img_sag.shape[1]),img_sag.shape[0]))
    mask_sag=np.array(np.max(mask1[k1:k2,max(k3-10,0):min(k4+10,219),max(k5-10,0):min(k6+10,379)],axis=0),dtype='uint8')
    mask_sag=cv2.resize(mask_sag,(round(2.8/1.65*mask_sag.shape[1]),mask_sag.shape[0]))
    plt.imshow(np.transpose(img_sag),cmap='gray')
    plot_outlines(np.transpose(mask_sag).T, lw=2, color='white')
    fig=plt.gcf()
    plt.show()
    fig.savefig('kuvat/{}_{}_sag.png'.format(organ_names[j],study_ids[i]),bbox_inches='tight')
    img_cor=np.mean(img[max(k1-10,0):min(k2+10,219),k3:k4,max(k5-10,0):min(k6+10,379)],axis=1)
    img_cor=cv2.resize(img_cor,(round(2.8/1.65*img_cor.shape[1]),img_cor.shape[0]))
    mask_cor=np.array(np.max(mask1[max(k1-10,0):min(k2+10,219),k3:k4,max(k5-10,0):min(k6+10,379)],axis=1),dtype='uint8')
    mask_cor=cv2.resize(mask_cor,(round(2.8/1.65*mask_cor.shape[1]),mask_cor.shape[0]))
    plt.imshow(np.transpose(img_cor),cmap='gray')
    plot_outlines(np.transpose(mask_cor).T, lw=2, color='white')
    fig=plt.gcf()
    fig.savefig('kuvat/{}_{}_cor.png'.format(organ_names[j],study_ids[i]),bbox_inches='tight')
    plt.show()
    img_trans=np.mean(img[max(k1-10,0):min(k2+10,219),max(k3-10,0):min(k4+10,219),k5:k6],axis=2)
    mask_trans=np.array(np.max(mask1[max(k1-10,0):min(k2+10,219),max(k3-10,0):min(k4+10,219),k5:k6],axis=2),dtype='uint8')
    plt.imshow(np.transpose(img_trans),cmap='gray')
    plot_outlines(np.transpose(mask_trans).T, lw=2, color='white')
    fig=plt.gcf()
    fig.savefig('kuvat/{}_{}_trans.png'.format(organ_names[j],study_ids[i]),bbox_inches='tight')
    plt.show()

In [ ]:
path_to_nifti='C:/Users/oonar/Documents/kov/files/img1.nii'
file=nib.load(path_to_nifti)
img=file.get_fdata()
img[img>50000]=50000
plt.imshow(np.transpose(np.mean(img==1,axis=1)),cmap='gray')

In [ ]:
index=1

folder_path='C:/Users/oonar/Documents/kov/files'
filenames=os.listdir(folder_path)
study_ids=[]
for i in range(len(filenames)):
    if 'img' in filenames[i]:
        number=int(filenames[i][3:-4])
        if index==1:
            if number not in [61,67,81,119,121]:
                study_ids.append(number)
        if index==2:
            study_ids.append(number)
        if index==3:
            if number not in [16,17,49,60,62,81,119,121,131,133]:
                study_ids.append(number)
        if index==4:
            if number not in [16,17,35,49,60,62,81,119,121,131,133]:
                study_ids.append(number)
        if index==5:
            if number not in [16,17,49,60,62,81,119,121,131,133]:
                study_ids.append(number)
        if index==6:
            if number not in [1,16,27,29,35,42,49,56,61,67,68,70,81,83,88,109,119,121,130,131,144,149,157]:
                study_ids.append(number)
        if index==7:
            if number not in [1,3,25,32,33,34,37,61,81,83,84,109,119,121,131,133,149]:
                study_ids.append(number)
        if index==8:
            if number not in [24,42,64,65]:
                study_ids.append(number)
        if index==9:
            if number not in [1,16,24,26,42,49,57,62,81,83,101,131,131,157]:
                study_ids.append(number)

In [ ]:
#classification
img_height=128
for view in ['sagittal','coronal','transaxial']:
    patientIndexes=[]
    slices=[]
    labels=[]
    for i in range(len(study_ids)):
        img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
        mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
        img=nib.load(img_path)
        img=img.get_fdata()
        mask=nib.load(mask_path)
        mask=mask.get_fdata()
        if index==3:
            mask[mask==4]=3
            mask[mask==5]=3
        img[img>50000]=50000
        img=img/50000
        if view=='sagittal':
            for j in range(img.shape[0]):
                patientIndexes.append(i)
                slices.append(preprocess_slice(img[j,:,:],img_height))
                labels.append(np.max(mask[j,:,:]==index))
        if view=='coronal':
            for j in range(img.shape[1]):
                patientIndexes.append(i)
                slices.append(preprocess_slice(img[:,j,:],img_height))
                labels.append(np.max(mask[:,j,:]==index))
        if view=='transaxial':
            for j in range(img.shape[2]):
                patientIndexes.append(i)
                slices.append(preprocess_slice(img[:,:,j],img_height))
                labels.append(np.max(mask[:,:,j]==index))

    for k in range(5):
        x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test=fiveFoldCrossValidation(patientIndexes,slices,labels,k)
        print(x_train.shape)
        numberOfEpochs=5
        startTime=perf_counter()
        predictions,trainPreds=predict(x_train,y_train,x_test,numberOfEpochs)
        endTime=perf_counter()
        processingTime=endTime-startTime
        predictions=postProcess_new(predictions,patientIndexes_test)
        trainPreds=postProcess_new(trainPreds,patientIndexes_train)
        np.savetxt('results/kov_{}_{}_{}_time.txt'.format(index,view,k),np.array([processingTime]))
        np.savetxt('results/kov_{}_{}_{}_predictions.txt'.format(index,view,k),predictions)
        np.savetxt('results/kov_{}_{}_{}_trainPreds.txt'.format(index,view,k),trainPreds)
        np.savetxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k),patientIndexes_test)
        np.savetxt('results/kov_{}_{}_{}_patientIndexes_train.txt'.format(index,view,k),patientIndexes_train)
        np.savetxt('results/kov_{}_{}_{}_y_test.txt'.format(index,view,k),y_test)
        np.savetxt('results/kov_{}_{}_{}_y_train.txt'.format(index,view,k),y_train)
        sliceIndexes=findPredictedSliceIndexes(predictions,trainPreds,y_train)
        np.savetxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,view,k),sliceIndexes)
        trainSliceIndexes=findPredictedSliceIndexes(trainPreds,trainPreds,y_train)
        np.savetxt('results/kov_{}_{}_{}_trainSliceIndexes.txt'.format(index,view,k),trainSliceIndexes)

In [ ]:
#compute the evaluation metrics
for view in ['sagittal','coronal','transaxial']:
    acc_list_1=[]
    sen_list_1=[]
    spe_list_1=[]
    pre_list_1=[]
    auc_list_1=[]
    for k in range(5):
        processingTimes=np.loadtxt('results/kov_{}_{}_{}_time.txt'.format(index,view,k))
        predictions=np.loadtxt('results/kov_{}_{}_{}_predictions.txt'.format(index,view,k))
        y_test=np.loadtxt('results/kov_{}_{}_{}_y_test.txt'.format(index,view,k))
        patientIndexes_test=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
        trainPreds=np.loadtxt('results/kov_{}_{}_{}_trainPreds.txt'.format(index,view,k))
        y_train=np.loadtxt('results/kov_{}_{}_{}_y_train.txt'.format(index,view,k))
        patientIndexes_train=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_train.txt'.format(index,view,k))
        acc_list,sen_list,spe_list,pre_list,auc_list=evaluatePreds(predictions,y_test,patientIndexes_test,trainPreds,y_train)
        acc_list_1=acc_list_1+acc_list
        sen_list_1=sen_list_1+sen_list
        spe_list_1=spe_list_1+spe_list
        pre_list_1=pre_list_1+pre_list
        auc_list_1=auc_list_1+auc_list
    if view=='coronal':
        lists_coronal=[acc_list_1,sen_list_1,spe_list_1,pre_list_1,auc_list_1]
    if view=='sagittal':
        lists_sagittal=[acc_list_1,sen_list_1,spe_list_1,pre_list_1,auc_list_1]
    if view=='transaxial':
        lists_transaxial=[acc_list_1,sen_list_1,spe_list_1,pre_list_1,auc_list_1]

for i in range(5):
    print(round(np.mean(lists_sagittal[i]),3)*100,'pmin',round(np.std(lists_sagittal[i]),3)*100,'&',round(np.mean(lists_coronal[i]),3)*100,'pmin',round(np.std(lists_coronal[i]),3)*100,'&',round(np.mean(lists_transaxial[i]),3)*100,'pmin',round(np.std(lists_transaxial[i]),3)*100)

In [ ]:
#evaluate 3d bounding boxes
dices=[]
volumes=[]
volumes_1=[]
sensitivity=[]
precision=[]
for i in range(len(study_ids)):
    mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    if index==3:
        mask[mask==4]=3
        mask[mask==5]=3
    mask=np.array(mask==index,dtype=int)
    k=i % 5
    mask_1=np.zeros((220,220,380))
    for view in ['sagittal','coronal','transaxial']:
        sliceIndexes=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,view,k))
        patientIndexes_test=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
        if view=='sagittal':
            for j in range(mask.shape[0]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[j,:,:]+=1
        if view=='coronal':
            for j in range(mask.shape[1]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[:,j,:]+=1
        if view=='transaxial':
            for j in range(mask.shape[2]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[:,:,j]+=1
    mask_1=np.array(mask_1>2.5,dtype=int)
    TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(mask_1>=0.5,dtype=int)))
    FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(mask_1<0.5,dtype=int)))
    FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(mask_1>=0.5,dtype=int)))
    TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(mask_1<0.5,dtype=int)))
    dice=2*TP/(2*TP+FN+FP)
    dices.append(dice)
    volumes.append((TP+FP)*0.165*0.165*0.28)
    volumes_1.append((TP+FP)/(TP+FP+FN+TN))
    sensitivity.append(TP/(TP+FN))
    if TP+FP>0:
        precision.append(TP/(TP+FP))
    print(i)

In [ ]:
print(round(np.mean(volumes)),'pmin',round(np.std(volumes)),'&',round(np.mean(volumes_1),3)*100,'pmin',round(np.std(volumes_1),3)*100,'&',round(np.mean(sensitivity),3)*100,'pmin',round(np.std(sensitivity),3)*100,'&',round(np.mean(precision),3)*100,'pmin',round(np.std(precision),3)*100)

In [ ]:
#segmentation
patientIndexes=[]
slices=[]
masks=[]
for view in ['transaxial']:#'sagittal','coronal',
    for k in range(1):
        trainSliceIndexes_1=np.loadtxt('results/kov_{}_{}_{}_trainSliceIndexes.txt'.format(index,'sagittal',k))
        patientIndexes_train_1=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_train.txt'.format(index,'sagittal',k))
        trainSliceIndexes_2=np.loadtxt('results/kov_{}_{}_{}_trainSliceIndexes.txt'.format(index,'coronal',k))
        patientIndexes_train_2=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_train.txt'.format(index,'coronal',k))
        trainSliceIndexes_3=np.loadtxt('results/kov_{}_{}_{}_trainSliceIndexes.txt'.format(index,'transaxial',k))
        patientIndexes_train_3=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_train.txt'.format(index,'transaxial',k))
        sliceIndexes_1=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,'sagittal',k))
        patientIndexes_test_1=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,'sagittal',k))
        sliceIndexes_2=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,'coronal',k))
        patientIndexes_test_2=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,'coronal',k))
        sliceIndexes_3=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,'transaxial',k))
        patientIndexes_test_3=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,'transaxial',k))
        for i in range(len(study_ids)):
            img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
            mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
            img=nib.load(img_path)
            img=img.get_fdata()
            mask=nib.load(mask_path)
            mask=mask.get_fdata()
            if index==3:
                mask[mask==4]=3
                mask[mask==5]=3
            img[img>50000]=50000
            img=img/50000
            mask_1=np.zeros((220,220,380))
            for j in range(mask_1.shape[0]):
                if i%5==k:
                    if sliceIndexes_1[patientIndexes_test_1==i][j]==1:
                        mask_1[j,:,:]+=1
                else:
                    if trainSliceIndexes_1[patientIndexes_train_1==i][j]==1:
                        mask_1[j,:,:]+=1
            for j in range(mask_1.shape[1]):
                if i%5==k:
                    if sliceIndexes_2[patientIndexes_test_2==i][j]==1:
                        mask_1[:,j,:]+=1
                else:
                    if trainSliceIndexes_2[patientIndexes_train_2==i][j]==1:
                        mask_1[:,j,:]+=1
            for j in range(mask_1.shape[2]):
                if i%5==k:
                    if sliceIndexes_3[patientIndexes_test_3==i][j]==1:
                        mask_1[:,:,j]+=1
                else:
                    if trainSliceIndexes_3[patientIndexes_train_3==i][j]==1:
                        mask_1[:,:,j]+=1
            mask_1=np.array(mask_1>2.5,dtype=int)
            if view=='sagittal':
                for j in range(mask_1.shape[0]):
                    if np.max(mask_1[j,:,:])==1:
                        for i1 in chooseIndices(np.max(mask_1[j,:,:],axis=1)):
                            for j1 in chooseIndices(np.max(mask_1[j,:,:],axis=0)):
                                if np.max(mask_1[j,i1:(i1+64),j1:(j1+64)])==1:
                                    patientIndexes.append(i)
                                    slices.append(img[j,i1:(i1+64),j1:(j1+64)].astype('float16'))
                                    masks.append(np.array(mask[j,i1:(i1+64),j1:(j1+64)]==index,dtype=int))
            if view=='coronal':
                for j in range(mask_1.shape[1]):
                    if np.max(mask_1[:,j,:])==1:
                        for i1 in chooseIndices(np.max(mask_1[:,j,:],axis=1)):
                            for j1 in chooseIndices(np.max(mask_1[:,j,:],axis=0)):
                                if np.max(mask_1[i1:(i1+64),j,j1:(j1+64)])==1:
                                    patientIndexes.append(i)
                                    slices.append(img[i1:(i1+64),j,j1:(j1+64)].astype('float16'))
                                    masks.append(np.array(mask[i1:(i1+64),j,j1:(j1+64)]==index,dtype=int))
            if view=='transaxial':
                for j in range(mask_1.shape[2]):
                    if np.max(mask_1[:,:,j])==1:
                        for i1 in chooseIndices(np.max(mask_1[:,:,j],axis=1)):
                            for j1 in chooseIndices(np.max(mask_1[:,:,j],axis=0)):
                                if np.max(mask_1[i1:(i1+64),j1:(j1+64),j])==1:
                                    patientIndexes.append(i)
                                    slices.append(img[i1:(i1+64),j1:(j1+64),j].astype('float16'))
                                    masks.append(np.array(mask[i1:(i1+64),j1:(j1+64),j]==index,dtype=int))
        x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test=fiveFoldCrossValidation(patientIndexes,slices,masks,k)
        print(x_train.shape)
        numberOfEpochs=5
        name='{}_{}_{}'.format(index,view,k)
        startTime=perf_counter()
        predictions,trainPreds=unet(x_train,y_train,x_test,numberOfEpochs,name)
        endTime=perf_counter()
        processingTime=endTime-startTime
        np.savetxt('results/kov_{}_seg_time.txt'.format(name),np.array([processingTime]))
        dices3d=computeDices(trainPreds,y_train,patientIndexes_train)
        print(np.mean(dices3d))
        dices3d=computeDices(predictions,y_test,patientIndexes_test)
        print(np.mean(dices3d))

In [ ]:
#compute the resulting Dice scores
segModel1_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',0))
segModel1_sag.compile(optimizer='adam',loss='Dice')
segModel2_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',1))
segModel2_sag.compile(optimizer='adam',loss='Dice')
segModel3_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',2))
segModel3_sag.compile(optimizer='adam',loss='Dice')
segModel4_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',3))
segModel4_sag.compile(optimizer='adam',loss='Dice')
segModel5_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',4))
segModel5_sag.compile(optimizer='adam',loss='Dice')

segModel1_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',0))
segModel1_cor.compile(optimizer='adam',loss='Dice')
segModel2_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',1))
segModel2_cor.compile(optimizer='adam',loss='Dice')
segModel3_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',2))
segModel3_cor.compile(optimizer='adam',loss='Dice')
segModel4_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',3))
segModel4_cor.compile(optimizer='adam',loss='Dice')
segModel5_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',4))
segModel5_cor.compile(optimizer='adam',loss='Dice')

segModel1_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',0))
segModel1_trans.compile(optimizer='adam',loss='Dice')
segModel2_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',1))
segModel2_trans.compile(optimizer='adam',loss='Dice')
segModel3_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',2))
segModel3_trans.compile(optimizer='adam',loss='Dice')
segModel4_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',3))
segModel4_trans.compile(optimizer='adam',loss='Dice')
segModel5_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',4))
segModel5_trans.compile(optimizer='adam',loss='Dice')

dices_cor=[]
dices_sag=[]
dices_trans=[]
dices_2_5d=[]
for i in range(len(study_ids)):
    print(i)
    img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
    mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
    img=nib.load(img_path)
    img=img.get_fdata()
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    if index==3:
        mask[mask==4]=3
        mask[mask==5]=3
    mask=np.array(mask==index,dtype=int)
    img[img>50000]=50000
    img=img/50000
    k=i % 5
    mask_1=np.zeros((220,220,380))
    for view in ['sagittal','coronal','transaxial']:
        sliceIndexes=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,view,k))
        patientIndexes_test=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
        if view=='sagittal':
            for j in range(mask.shape[0]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[j,:,:]+=1
        if view=='coronal':
            for j in range(mask.shape[1]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[:,j,:]+=1
        if view=='transaxial':
            for j in range(mask.shape[2]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[:,:,j]+=1
    mask_1=np.array(mask_1>2.5,dtype=int)
    newMask_2_5d=np.zeros(img.shape)
    for view in ['coronal','sagittal','transaxial']:
        newMask=np.zeros(img.shape)
        denominators=np.zeros(img.shape)
        if view=='sagittal':
            for j in range(mask_1.shape[0]):
                if np.max(mask_1[j,:,:])==1:
                    for i1 in chooseIndices(np.max(mask_1[j,:,:],axis=1)):
                        for j1 in chooseIndices(np.max(mask_1[j,:,:],axis=0)):
                            if np.max(mask_1[j,i1:(i1+64),j1:(j1+64)])==1:
                                slice=img[j,i1:(i1+64),j1:(j1+64)].astype('float16')
                                if k==0:
                                    pred=segModel1_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==1:
                                    pred=segModel2_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==2:
                                    pred=segModel3_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==3:
                                    pred=segModel4_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==4:
                                    pred=segModel5_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                newMask[j,i1:(i1+64),j1:(j1+64)]+=pred
                                denominators[j,i1:(i1+64),j1:(j1+64)]+=1
        if view=='coronal':
            for j in range(mask_1.shape[1]):
                if np.max(mask_1[:,j,:])==1:
                    for i1 in chooseIndices(np.max(mask_1[:,j,:],axis=1)):
                        for j1 in chooseIndices(np.max(mask_1[:,j,:],axis=0)):
                            if np.max(mask_1[i1:(i1+64),j,j1:(j1+64)])==1:
                                slice=img[i1:(i1+64),j,j1:(j1+64)].astype('float16')
                                if k==0:
                                    pred=segModel1_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==1:
                                    pred=segModel2_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==2:
                                    pred=segModel3_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==3:
                                    pred=segModel4_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==4:
                                    pred=segModel5_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                newMask[i1:(i1+64),j,j1:(j1+64)]+=pred
                                denominators[i1:(i1+64),j,j1:(j1+64)]+=1
        if view=='transaxial':
            for j in range(mask_1.shape[2]):
                if np.max(mask_1[:,:,j])==1:
                    for i1 in chooseIndices(np.max(mask_1[:,:,j],axis=1)):
                        for j1 in chooseIndices(np.max(mask_1[:,:,j],axis=0)):
                            if np.max(mask_1[i1:(i1+64),j1:(j1+64),j])==1:
                                slice=img[i1:(i1+64),j1:(j1+64),j].astype('float16')
                                if k==0:
                                    pred=segModel1_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==1:
                                    pred=segModel2_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==2:
                                    pred=segModel3_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==3:
                                    pred=segModel4_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==4:
                                    pred=segModel5_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                newMask[i1:(i1+64),j1:(j1+64),j]+=pred
                                denominators[i1:(i1+64),j1:(j1+64),j]+=1
        denominators[denominators==0]=1
        newMask=newMask/denominators
        newMask=np.array(newMask>0.5,dtype=int)
        TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
        FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
        FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
        TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
        dice=2*TP/(2*TP+FN+FP)
        print(dice)
        if view=='coronal':
            dices_cor.append(dice)
        if view=='sagittal':
            dices_sag.append(dice)
        if view=='transaxial':
            dices_trans.append(dice)
        newMask_2_5d+=newMask
    newMask=np.array(newMask_2_5d>1.5,dtype=int)
    TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
    FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
    FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
    TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
    dice=2*TP/(2*TP+FN+FP)
    dices_2_5d.append(dice)
    print(dice)
np.savetxt('results/kov_{}_dices_cor.txt'.format(index),dices_cor)
np.savetxt('results/kov_{}_dices_sag.txt'.format(index),dices_sag)
np.savetxt('results/kov_{}_dices_trans.txt'.format(index),dices_trans)
np.savetxt('results/kov_{}_dices_2_5d.txt'.format(index),dices_2_5d)  

In [ ]:
#brain 78, 31.1
#aorta 79, 48.5
#heart 86, 53.7
#myocardium 31, 29.4
#left ventricle 24, 27.1
#kidneys 43, 31.0
#liver 132, 42.1
#lungs 241, 27.1

In [ ]:
print(round(np.mean(dices_sag),3)*100,'pmin',round(np.std(dices_sag),3)*100,'&',round(np.mean(dices_cor),3)*100,'pmin',round(np.std(dices_cor),3)*100,'&',round(np.mean(dices_trans),3)*100,'pmin',round(np.std(dices_trans),3)*100,'&',round(np.mean(dices_2_5d),3)*100,'pmin',round(np.std(dices_2_5d),3)*100)

In [ ]:
i=0
img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
img=nib.load(img_path)
img=img.get_fdata()
mask=nib.load(mask_path)
mask=mask.get_fdata()
if index==3:
    mask[mask==4]=3
    mask[mask==5]=3
img[img>50000]=50000
img=img/50000

In [ ]:
img_i=np.transpose(np.mean(img,axis=1))
img_i=cv2.resize(img_i,(220,round(2.80/1.65*380)))
plt.imshow(img_i,cmap='gray')

In [ ]:
kuv=np.ones((755,330))
for k in range(110):
    img_i=np.transpose(img[:,2*k,:])
    img_i=cv2.resize(img_i,(220,round(2.80/1.65*380)))
    kuv[(110-k):(645+110-k),k:(220+k)]+=img_i
kuv=kuv/np.max(kuv)
for k in range(110):
    kuv[0:(110-k),0:k]=1
    kuv[(645+110-k):755,(220+k):330]=1
    kuv[(109-k):(112-k),(219+k):(222+k)]=1
kuv[110:755,219:222]=1
kuv[109:112,0:221]=1
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig1.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(7):
    img_i=np.transpose(img[30*k,:,:])
    img_i=cv2.resize(img_i,(220,round(2.80/1.65*380)))
    img_i[0:645,0:5]=1
    img_i[0:5,0:220]=1
    for i in range(645):
        for j in range(110):
            kuv[i+110-j,j+36*k]=img_i[i,2*j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
#fig.savefig('kfig2.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(7):
    img_i=np.transpose(img[30*k,:,:])
    img_i=cv2.resize(img_i,(220,220))
    img_i[0:220,0:5]=1
    img_i[0:5,0:220]=1
    for i in range(220):
        for j in range(110):
            kuv[i+110-j,j+36*k]=img_i[i,2*j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig25.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(7):
    img_i=np.transpose(img[:,219-27*k,:])
    img_i=cv2.resize(img_i,(220,round(2.80/1.65*380)))
    img_i[0:645,217:220]=1
    img_i[0:3,0:220]=1
    kuv[18*k:(18*k+645),6*18-18*k:(220+6*18-18*k)]=img_i
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig3.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(7):
    img_i=np.transpose(img[:,219-27*k,:])
    img_i=cv2.resize(img_i,(220,220))
    img_i[0:220,217:220]=1
    img_i[0:3,0:220]=1
    kuv[18*k:(18*k+220),6*18-18*k:(220+6*18-18*k)]=img_i
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig26.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(14):
    img_i=np.flip(np.transpose(img[:,:,379-27*k]),0)
    img_i[0:220,216:220]=1
    img_i[214:220,0:220]=1
    for i in range(110):
        for j in range(220):
            kuv[i+645-49*k,j+110-i]=img_i[2*i,j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig4.png',bbox_inches='tight')

In [ ]:
i=0
mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
mask=nib.load(mask_path)
mask=mask.get_fdata()
if index==3:
    mask[mask==4]=3
    mask[mask==5]=3
mask=np.array(mask==index,dtype=int)
k=i % 5
mask_1=np.zeros((220,220,380))
for view in ['sagittal','coronal','transaxial']:
    sliceIndexes=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,view,k))
    patientIndexes_test=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
    if view=='sagittal':
        for j in range(mask.shape[0]):
            if sliceIndexes[patientIndexes_test==i][j]==1:
                mask_1[j,:,:]+=1
    if view=='coronal':
        for j in range(mask.shape[1]):
            if sliceIndexes[patientIndexes_test==i][j]==1:
                mask_1[:,j,:]+=1
    if view=='transaxial':
        for j in range(mask.shape[2]):
            if sliceIndexes[patientIndexes_test==i][j]==1:
                mask_1[:,:,j]+=1
mask_1=np.array(mask_1>2.5,dtype=int)

In [ ]:
kuv=np.ones((755,330))
for k in range(110):
    img_i=np.transpose(mask_1[:,2*k,:])
    img_i=cv2.resize(np.array(img_i,dtype='uint8'),(220,round(2.80/1.65*380)))
    kuv[(110-k):(645+110-k),k:(220+k)]+=img_i
kuv=kuv/np.max(kuv)
kuv=0.5*np.array(kuv>0.1)+0.5*kuv
kuv[152:155,91:165]=1
kuv[155:237,162:165]=1
for k in range(48):
    kuv[(152-k):(155-k),(162+k):(165+k)]=1
for k in range(110):
    kuv[0:(110-k),0:k]=1
    kuv[(645+110-k):755,(220+k):330]=1
    kuv[(109-k):(112-k),(219+k):(222+k)]=1
kuv[110:755,219:222]=1
kuv[109:112,0:221]=1
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig8.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((255,230))
for k in range(7):
    if np.max(mask_1[30*k,:,:])==0:
        img_i=np.zeros((30,30))
    else:
        img_i=np.zeros((30,30))
        img_i[3:27,3:27]=1
    for i in range(30):
        for j in range(15):
            kuv[i+110-j,j+19*k+30]=img_i[i,2*j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig5.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((255,330))
for k in range(7):
    if np.max(mask_1[:,219-27*k,:])==0:
        img_i=np.zeros((30,30))
    else:
        img_i=np.zeros((30,30))
        img_i[3:27,3:27]=1
    kuv[18*k+100:(18*k+30)+100,6*18-18*k+70:(30+6*18-18*k)+70]=img_i
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig6.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,230))
for k in range(14):
    if np.max(mask_1[:,:,379-27*k])==0:
        img_i=np.zeros((30,30))
    else:
        img_i=np.zeros((30,30))
        img_i[3:27,3:27]=1
    for i in range(15):
        for j in range(30):
            kuv[i+345-19*k+30,j-i+30]=img_i[2*i,j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig7.png',bbox_inches='tight')

In [ ]:
sag_img_list=[]
sag_mask_list=[]
cor_img_list=[]
cor_mask_list=[]
trans_img_list=[]
trans_mask_list=[]
for i in range(1):
    print(i)
    img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
    mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
    img=nib.load(img_path)
    img=img.get_fdata()
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    if index==3:
        mask[mask==4]=3
        mask[mask==5]=3
    mask=np.array(mask==index,dtype=int)
    img[img>50000]=50000
    img=img/50000
    mask_1=np.zeros((220,220,380))
    k=i % 5
    mask_1=np.zeros((220,220,380))
    for view in ['sagittal','coronal','transaxial']:
        sliceIndexes=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,view,k))
        patientIndexes_test=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
        if view=='sagittal':
            for j in range(mask.shape[0]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[j,:,:]+=1
        if view=='coronal':
            for j in range(mask.shape[1]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[:,j,:]+=1
        if view=='transaxial':
            for j in range(mask.shape[2]):
                if sliceIndexes[patientIndexes_test==i][j]==1:
                    mask_1[:,:,j]+=1
    mask_1=np.array(mask_1>2.5,dtype=int)
    newMask_2_5d=np.zeros(img.shape)
    for view in ['coronal','sagittal','transaxial']:
        newMask=np.zeros(img.shape)
        denominators=np.zeros(img.shape)
        if view=='sagittal':
            for j in range(mask_1.shape[0]):
                if np.max(mask_1[j,:,:])==1:
                    for i1 in chooseIndices(np.max(mask_1[j,:,:],axis=1)):
                        for j1 in chooseIndices(np.max(mask_1[j,:,:],axis=0)):
                            if np.max(mask_1[j,i1:(i1+64),j1:(j1+64)])==1:
                                slice=img[j,i1:(i1+64),j1:(j1+64)].astype('float16')
                                sag_img_list.append(slice)
                                sag_mask_list.append(mask[j,i1:(i1+64),j1:(j1+64)])
        if view=='coronal':
            for j in range(mask_1.shape[1]):
                if np.max(mask_1[:,j,:])==1:
                    for i1 in chooseIndices(np.max(mask_1[:,j,:],axis=1)):
                        for j1 in chooseIndices(np.max(mask_1[:,j,:],axis=0)):
                            if np.max(mask_1[i1:(i1+64),j,j1:(j1+64)])==1:
                                slice=img[i1:(i1+64),j,j1:(j1+64)].astype('float16')
                                cor_img_list.append(slice)
                                cor_mask_list.append(mask[i1:(i1+64),j,j1:(j1+64)])
        if view=='transaxial':
            for j in range(mask_1.shape[2]):
                if np.max(mask_1[:,:,j])==1:
                    for i1 in chooseIndices(np.max(mask_1[:,:,j],axis=1)):
                        for j1 in chooseIndices(np.max(mask_1[:,:,j],axis=0)):
                            if np.max(mask_1[i1:(i1+64),j1:(j1+64),j])==1:
                                slice=img[i1:(i1+64),j1:(j1+64),j].astype('float16')
                                trans_img_list.append(slice)
                                trans_mask_list.append(mask[i1:(i1+64),j1:(j1+64),j])

In [ ]:
kuv=np.ones((455,330))
for k in range(7):
    img_i=np.transpose(sag_img_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    img_i=img_i/np.max(img_i)
    img_i[0:220,0:5]=1
    img_i[0:5,0:220]=1
    for i in range(220):
        for j in range(110):
            kuv[i+110-j,j+36*k]=img_i[i,2*j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig9.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(7):
    img_i=np.transpose(cor_img_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    img_i=img_i/np.max(img_i)
    img_i[0:220,217:220]=1
    img_i[0:3,0:220]=1
    kuv[18*k:(18*k+220),6*18-18*k:(220+6*18-18*k)]=img_i
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig10.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(7):
    img_i=np.transpose(trans_img_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    img_i=img_i/np.max(img_i)
    img_i[0:220,215:220]=1
    img_i[215:220,0:220]=1
    for i in range(110):
        for j in range(220):
            kuv[i+273-39*k,j+110-i]=img_i[2*i,j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig11.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(7):
    img_i=np.transpose(sag_mask_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    if np.max(img_i)>0:
        img_i=img_i/np.max(img_i)
    img_i[0:220,0:5]=1
    img_i[0:5,0:220]=1
    for i in range(220):
        for j in range(110):
            kuv[i+110-j,j+36*k]=img_i[i,2*j]
kuv[5:220,324:327]=0
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig12.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(7):
    img_i=np.transpose(cor_mask_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    img_i=img_i/np.max(img_i)
    img_i[0:220,217:220]=1
    img_i[0:3,0:220]=1
    kuv[18*k:(18*k+220),6*18-18*k:(220+6*18-18*k)]=img_i
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig13.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(7):
    img_i=np.transpose(trans_mask_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    if np.max(img_i)>0:
        img_i=img_i/np.max(img_i)
    img_i[0:220,215:220]=1
    img_i[215:220,0:220]=1
    for i in range(110):
        for j in range(220):
            kuv[i+273-39*k,j+110-i]=img_i[2*i,j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig14.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(110):
    img_i=np.transpose(mask[:,2*k,:])
    img_i=cv2.resize(np.array(img_i,dtype='uint8'),(220,round(2.80/1.65*380)))
    kuv[(110-k):(645+110-k),k:(220+k)]+=img_i
kuv=kuv/np.max(kuv)
for k in range(110):
    kuv[0:(110-k),0:k]=1
    kuv[(645+110-k):755,(220+k):330]=1
    kuv[(109-k):(112-k),(219+k):(222+k)]=1
kuv[110:755,219:222]=1
kuv[109:112,0:221]=1
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig18.png',bbox_inches='tight')

In [ ]:
from scipy.ndimage import binary_dilation, binary_erosion
structure=np.zeros((3,3,3))
structure[1,1,1]=1
newMask=binary_erosion(mask,structure)
kuv=np.ones((755,330))
for k in range(110):
    img_i=np.transpose(newMask[:,2*k,:])
    img_i=cv2.resize(np.array(img_i,dtype='uint8'),(220,round(2.80/1.65*380)))
    kuv[(110-k):(645+110-k),k:(220+k)]+=img_i
kuv=kuv/np.max(kuv)
for k in range(110):
    kuv[0:(110-k),0:k]=1
    kuv[(645+110-k):755,(220+k):330]=1
    kuv[(109-k):(112-k),(219+k):(222+k)]=1
kuv[110:755,219:222]=1
kuv[109:112,0:221]=1
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig17.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(12,13):
    img_i=np.flip(np.transpose(mask_1[:,:,379-27*k]),0)
    img_i[0:220,216:220]=1
    img_i[214:220,0:220]=1
    for i in range(110):
        for j in range(220):
            kuv[i+645-49*k,j+110-i]=img_i[2*i,j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig19.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((755,330))
for k in range(14):
    img_i=np.flip(np.transpose(mask_1[:,:,379-27*k]),0)
    img_i[0:220,216:220]=1
    img_i[214:220,0:220]=1
    for i in range(110):
        for j in range(220):
            kuv[i+645-49*k,j+110-i]=img_i[2*i,j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig20.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(1):
    img_i=np.transpose(img[:,90,:])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    img_i=img_i/np.max(img_i)
    img_i[0:220,0:5]=1
    img_i[0:5,0:220]=1
    for i in range(220):
        for j in range(110):
            kuv[i+110-j,j+36*k]=img_i[i,2*j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig21.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(1):
    img_i=np.transpose(cor_img_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    img_i=img_i/np.max(img_i)
    img_i[0:220,0:5]=1
    img_i[0:5,0:220]=1
    for i in range(220):
        for j in range(110):
            kuv[i+110-j,j+36*k]=img_i[i,2*j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig22.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((455,330))
for k in range(1):
    img_i=np.transpose(cor_mask_list[3*k])
    img_i=cv2.resize(np.array(100*img_i,dtype='uint8'),(220,220))
    img_i=img_i/np.max(img_i)
    img_i[0:220,0:5]=1
    img_i[0:5,0:220]=1
    for i in range(220):
        for j in range(110):
            kuv[i+110-j,j+36*k]=img_i[i,2*j]
plt.imshow(kuv,cmap='gray')
plt.axis('off')
fig=plt.gcf()
fig.savefig('kfig23.png',bbox_inches='tight')

In [ ]:
def mobilenet_block(x,filters,strides):
    
    x = tf.keras.layers.DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    x = tf.keras.layers.Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    return x

def predict_mobilenet(x_train,y_train,x_test,numberOfEpochs):

    if len(x_train[0].shape)==3:
        input_shape=x_train[0].shape
    else:
        input_shape=(128,128,1)

    input = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    # main part of the model
    x = mobilenet_block(x, filters = 64, strides = 1)
    x = mobilenet_block(x, filters = 128, strides = 2)
    x = mobilenet_block(x, filters = 128, strides = 1)
    x = mobilenet_block(x, filters = 256, strides = 2)
    x = mobilenet_block(x, filters = 256, strides = 1)
    x = mobilenet_block(x, filters = 512, strides = 2) 
    x = tf.keras.layers.AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
    x = tf.keras.layers.GlobalAveragePooling2D (data_format='channels_first')(x)   
    output = tf.keras.layers.Dense (units = 1, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs=input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    history=model.fit(x=x_train,y=y_train,epochs=numberOfEpochs,shuffle=True)
    predictions=model.predict(x_test)[:,0]
    trainPreds=model.predict(x_train)[:,0]
    plt.plot(range(len(history.history['loss'])),history.history['loss'],color='blue')

    return([predictions,trainPreds])

In [ ]:
#classification
img_height=128
for view in ['transaxial']:
    patientIndexes=[]
    slices=[]
    labels=[]
    for i in range(len(study_ids)):
        img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
        mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
        img=nib.load(img_path)
        img=img.get_fdata()
        mask=nib.load(mask_path)
        mask=mask.get_fdata()
        if index==3:
            mask[mask==4]=3
            mask[mask==5]=3
        img[img>50000]=50000
        img=img/50000
        if view=='sagittal':
            for j in range(img.shape[0]):
                patientIndexes.append(i)
                slices.append(preprocess_slice(img[j,:,:],img_height))
                labels.append(np.max(mask[j,:,:]==index))
        if view=='coronal':
            for j in range(img.shape[1]):
                patientIndexes.append(i)
                slices.append(preprocess_slice(img[:,j,:],img_height))
                labels.append(np.max(mask[:,j,:]==index))
        if view=='transaxial':
            for j in range(img.shape[2]):
                patientIndexes.append(i)
                slices.append(preprocess_slice(img[:,:,j],img_height))
                labels.append(np.max(mask[:,:,j]==index))

    for k in range(1):
        x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test=fiveFoldCrossValidation(patientIndexes,slices,labels,k)
        print(x_train.shape)
        numberOfEpochs=5
        startTime=perf_counter()
        predictions,trainPreds=predict_mobilenet(x_train,y_train,x_test,numberOfEpochs)
        endTime=perf_counter()
        processingTime=endTime-startTime
        predictions=postProcess_new(predictions,patientIndexes_test)
        trainPreds=postProcess_new(trainPreds,patientIndexes_train)
        np.savetxt('results/mobilenet_{}_{}_{}_time.txt'.format(index,view,k),np.array([processingTime]))
        np.savetxt('results/mobilenet_{}_{}_{}_predictions.txt'.format(index,view,k),predictions)
        np.savetxt('results/mobilenet_{}_{}_{}_trainPreds.txt'.format(index,view,k),trainPreds)
        np.savetxt('results/mobilenet_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k),patientIndexes_test)
        np.savetxt('results/mobilenet_{}_{}_{}_patientIndexes_train.txt'.format(index,view,k),patientIndexes_train)
        np.savetxt('results/mobilenet_{}_{}_{}_y_test.txt'.format(index,view,k),y_test)
        np.savetxt('results/mobilenet_{}_{}_{}_y_train.txt'.format(index,view,k),y_train)
        sliceIndexes=findPredictedSliceIndexes(predictions,trainPreds,y_train)
        np.savetxt('results/mobilenet_{}_{}_{}_sliceIndexes.txt'.format(index,view,k),sliceIndexes)
        trainSliceIndexes=findPredictedSliceIndexes(trainPreds,trainPreds,y_train)
        np.savetxt('results/mobilenet_{}_{}_{}_trainSliceIndexes.txt'.format(index,view,k),trainSliceIndexes)

In [ ]:
#compute the evaluation metrics
view='transaxial'
k=0
processingTimes=np.loadtxt('results/kov_{}_{}_{}_time.txt'.format(index,view,k))
predictions=np.loadtxt('results/kov_{}_{}_{}_predictions.txt'.format(index,view,k))
y_test=np.loadtxt('results/kov_{}_{}_{}_y_test.txt'.format(index,view,k))
patientIndexes_test=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
trainPreds=np.loadtxt('results/kov_{}_{}_{}_trainPreds.txt'.format(index,view,k))
y_train=np.loadtxt('results/kov_{}_{}_{}_y_train.txt'.format(index,view,k))
patientIndexes_train=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_train.txt'.format(index,view,k))
acc_list,sen_list,spe_list,pre_list,auc_list=evaluatePreds(predictions,y_test,patientIndexes_test,trainPreds,y_train)
print(processingTimes)

processingTimes=np.loadtxt('results/mobilenet_{}_{}_{}_time.txt'.format(index,view,k))
predictions=np.loadtxt('results/mobilenet_{}_{}_{}_predictions.txt'.format(index,view,k))
y_test=np.loadtxt('results/mobilenet_{}_{}_{}_y_test.txt'.format(index,view,k))
patientIndexes_test=np.loadtxt('results/mobilenet_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
trainPreds=np.loadtxt('results/mobilenet_{}_{}_{}_trainPreds.txt'.format(index,view,k))
y_train=np.loadtxt('results/mobilenet_{}_{}_{}_y_train.txt'.format(index,view,k))
patientIndexes_train=np.loadtxt('results/mobilenet_{}_{}_{}_patientIndexes_train.txt'.format(index,view,k))
acc_list1,sen_list1,spe_list1,pre_list1,auc_list1=evaluatePreds(predictions,y_test,patientIndexes_test,trainPreds,y_train)
print(processingTimes)

print(np.mean(np.array(auc_list)))
print(np.std(np.array(auc_list)))
print(np.mean(np.array(auc_list1)))
print(np.std(np.array(auc_list1)))
print(wilcoxon(np.array(auc_list),np.array(auc_list1)))

In [ ]:
for index in range(1,9):

    folder_path='C:/Users/oonar/Documents/kov/files'
    filenames=os.listdir(folder_path)
    study_ids=[]
    for i in range(len(filenames)):
        if 'img' in filenames[i]:
            number=int(filenames[i][3:-4])
            if index==1:
                if number not in [61,67,81,119,121]:
                    study_ids.append(number)
            if index==2:
                study_ids.append(number)
            if index==3:
                if number not in [16,17,49,60,62,81,119,121,131,133]:
                    study_ids.append(number)
            if index==4:
                if number not in [16,17,35,49,60,62,81,119,121,131,133]:
                    study_ids.append(number)
            if index==5:
                if number not in [16,17,49,60,62,81,119,121,131,133]:
                    study_ids.append(number)
            if index==6:
                if number not in [1,16,27,29,35,42,49,56,61,67,68,70,81,83,88,109,119,121,130,131,144,149,157]:
                    study_ids.append(number)
            if index==7:
                if number not in [1,3,25,32,33,34,37,61,81,83,84,109,119,121,131,133,149]:
                    study_ids.append(number)
            if index==8:
                if number not in [24,42,64,65]:
                    study_ids.append(number)
   
    #compute the resulting Dice scores
    segModel1_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',0))
    segModel1_sag.compile(optimizer='adam',loss='Dice')
    segModel2_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',1))
    segModel2_sag.compile(optimizer='adam',loss='Dice')
    segModel3_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',2))
    segModel3_sag.compile(optimizer='adam',loss='Dice')
    segModel4_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',3))
    segModel4_sag.compile(optimizer='adam',loss='Dice')
    segModel5_sag=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'sagittal',4))
    segModel5_sag.compile(optimizer='adam',loss='Dice')

    segModel1_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',0))
    segModel1_cor.compile(optimizer='adam',loss='Dice')
    segModel2_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',1))
    segModel2_cor.compile(optimizer='adam',loss='Dice')
    segModel3_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',2))
    segModel3_cor.compile(optimizer='adam',loss='Dice')
    segModel4_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',3))
    segModel4_cor.compile(optimizer='adam',loss='Dice')
    segModel5_cor=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'coronal',4))
    segModel5_cor.compile(optimizer='adam',loss='Dice')

    segModel1_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',0))
    segModel1_trans.compile(optimizer='adam',loss='Dice')
    segModel2_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',1))
    segModel2_trans.compile(optimizer='adam',loss='Dice')
    segModel3_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',2))
    segModel3_trans.compile(optimizer='adam',loss='Dice')
    segModel4_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',3))
    segModel4_trans.compile(optimizer='adam',loss='Dice')
    segModel5_trans=keras.models.load_model('unet_{}_{}_{}.keras'.format(index,'transaxial',4))
    segModel5_trans.compile(optimizer='adam',loss='Dice')

    empty_header=nib.Nifti1Header()
    affine_matrix=np.eye(4)
    affine_matrix[0,0]=-1.65
    affine_matrix[1,1]=-1.65
    affine_matrix[2,2]=-2.8

    organ_names=['-','brain','aorta','heart','myocardium','left_ventricle','kidneys','liver','lungs']
    indicators=[['a','b','a','b','b','b','a','a'],
            ['b','b','a','a','a','a','b','b'],
            ['a','b','b','a','b','a','a','b'],
            ['a','a','a','b','b','b','b','a'],
            ['b','a','b','b','b','b','a','a'],
            ['b','b','b','a','a','a','b','b'],
            ['b','a','a','b','b','b','b','b'],
            ['a','b','a','a','b','a','a','a'],
            ['b','a','b','a','a','b','a','a'],
            ['a','b','a','a','b','b','b','b']]
    for k in range(10):
        for i in range(len(study_ids)):
            if study_ids[i]==[2,4,5,6,7,8,10,11,12,13][k]:
                img_path='C:/Users/oonar/Documents/kov/files/img{}.nii'.format(study_ids[i])
                mask_path='C:/Users/oonar/Documents/kov/files/mask{}.nii'.format(study_ids[i])
                img=nib.load(img_path)
                img=img.get_fdata()
                mask=nib.load(mask_path)
                mask=mask.get_fdata()
                if index==3:
                    mask[mask==4]=3
                    mask[mask==5]=3
                mask=np.array(mask==index,dtype=int)
                img[img>50000]=50000
                img=img/50000
                new_img=nib.Nifti1Image(img,affine_matrix,empty_header)
                nib.save(new_img,'C:/Users/oonar/Documents/kov_testi/id{}/id{}_pet_image.nii'.format(study_ids[i],study_ids[i]))
                indic=indicators[k][index-1]
                if indic=='a':
                    indic1='b'
                else:
                    indic1='a'
                new_img=nib.Nifti1Image(mask,affine_matrix,empty_header)
                nib.save(new_img,'C:/Users/oonar/Documents/kov_testi/id{}/id{}_{}_{}.nii'.format(study_ids[i],study_ids[i],organ_names[index],indic1))
                k=i % 5
                mask_1=np.zeros((220,220,380))
                for view in ['sagittal','coronal','transaxial']:
                    sliceIndexes=np.loadtxt('results/kov_{}_{}_{}_sliceIndexes.txt'.format(index,view,k))
                    patientIndexes_test=np.loadtxt('results/kov_{}_{}_{}_patientIndexes_test.txt'.format(index,view,k))
                    if view=='sagittal':
                        for j in range(mask.shape[0]):
                            if sliceIndexes[patientIndexes_test==i][j]==1:
                                mask_1[j,:,:]+=1
                    if view=='coronal':
                        for j in range(mask.shape[1]):
                            if sliceIndexes[patientIndexes_test==i][j]==1:
                                mask_1[:,j,:]+=1
                    if view=='transaxial':
                        for j in range(mask.shape[2]):
                            if sliceIndexes[patientIndexes_test==i][j]==1:
                                mask_1[:,:,j]+=1
                mask_1=np.array(mask_1>2.5,dtype=int)
                newMask_2_5d=np.zeros(img.shape)
                for view in ['coronal','sagittal','transaxial']:
                    newMask=np.zeros(img.shape)
                    denominators=np.zeros(img.shape)
                    if view=='sagittal':
                        for j in range(mask_1.shape[0]):
                            if np.max(mask_1[j,:,:])==1:
                                for i1 in chooseIndices(np.max(mask_1[j,:,:],axis=1)):
                                    for j1 in chooseIndices(np.max(mask_1[j,:,:],axis=0)):
                                        if np.max(mask_1[j,i1:(i1+64),j1:(j1+64)])==1:
                                            slice=img[j,i1:(i1+64),j1:(j1+64)].astype('float16')
                                            if k==0:
                                                pred=segModel1_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==1:
                                                pred=segModel2_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==2:
                                                pred=segModel3_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==3:
                                                pred=segModel4_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==4:
                                                pred=segModel5_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            newMask[j,i1:(i1+64),j1:(j1+64)]+=pred
                                            denominators[j,i1:(i1+64),j1:(j1+64)]+=1
                    if view=='coronal':
                        for j in range(mask_1.shape[1]):
                            if np.max(mask_1[:,j,:])==1:
                                for i1 in chooseIndices(np.max(mask_1[:,j,:],axis=1)):
                                    for j1 in chooseIndices(np.max(mask_1[:,j,:],axis=0)):
                                        if np.max(mask_1[i1:(i1+64),j,j1:(j1+64)])==1:
                                            slice=img[i1:(i1+64),j,j1:(j1+64)].astype('float16')
                                            if k==0:
                                                pred=segModel1_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==1:
                                                pred=segModel2_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==2:
                                                pred=segModel3_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==3:
                                                pred=segModel4_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==4:
                                                pred=segModel5_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            newMask[i1:(i1+64),j,j1:(j1+64)]+=pred
                                            denominators[i1:(i1+64),j,j1:(j1+64)]+=1
                    if view=='transaxial':
                        for j in range(mask_1.shape[2]):
                            if np.max(mask_1[:,:,j])==1:
                                for i1 in chooseIndices(np.max(mask_1[:,:,j],axis=1)):
                                    for j1 in chooseIndices(np.max(mask_1[:,:,j],axis=0)):
                                        if np.max(mask_1[i1:(i1+64),j1:(j1+64),j])==1:
                                            slice=img[i1:(i1+64),j1:(j1+64),j].astype('float16')
                                            if k==0:
                                                pred=segModel1_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==1:
                                                pred=segModel2_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==2:
                                                pred=segModel3_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==3:
                                                pred=segModel4_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            if k==4:
                                                pred=segModel5_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                            newMask[i1:(i1+64),j1:(j1+64),j]+=pred
                                            denominators[i1:(i1+64),j1:(j1+64),j]+=1
                    denominators[denominators==0]=1
                    newMask=newMask/denominators
                    newMask=np.array(newMask>0.5,dtype=int)
                    newMask_2_5d+=newMask
                newMask=np.array(newMask_2_5d>1.5,dtype=int)
                TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
                FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
                FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
                TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
                dice=2*TP/(2*TP+FN+FP)
                print(dice)
                new_img=nib.Nifti1Image(newMask,affine_matrix,empty_header)
                nib.save(new_img,'C:/Users/oonar/Documents/kov_testi/id{}/id{}_{}_{}.nii'.format(study_ids[i],study_ids[i],organ_names[index],indic))